<a href="https://colab.research.google.com/github/gmoulantz2/Translate-PDF-files-with-Hugginface-and-PyPDF2/blob/main/Translation_with_Hugginface_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers sentencepiece sacremoses 

In [1]:
from transformers import pipeline

In [ ]:

from transformers.models.mgp_str.processing_mgp_str import AutoTokenizer
from transformers.models.auto.modeling_auto import AutoModelForSeq2SeqLM


1) Translator Class, available languages: English, French, German.

In [23]:
class Translator:

  def __init__(self):

    self.model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
    self.tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
    self.languages = ['en', 'fr', 'de']

  def translate(self, text, src_lang, tgt_lang):

    if src_lang not in self.languages:
      raise RuntimeError('Source language not supported.')
    if tgt_lang not in self.languages:
      raise RuntimeError('Target language not supported')

    model_name = 'Helsinki-NLP/opus-mt-' + src_lang + '-' + tgt_lang
    self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    inputs = self.tokenizer.encode(text, return_tensors = 'pt')
    tokens = self.model.generate(inputs, max_length = 512)
    decoded = self.tokenizer.decode(tokens[0], skip_special_tokens = True)

    return decoded

    
    


In [24]:
translator_object = Translator()

Example: Translating 'bonjour' in English.

In [47]:
translator_object.translate('bonjour', 'fr', 'en')

'Hello.'

2) Function that converts a pdf file to text format. The output will be used for translation.

In [ ]:
pip install PyPDF2

In [28]:
import PyPDF2

def convert_pdf_to_txt(input_path, output_path):
    
    with open(input_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        with open(output_path, 'w') as txt_file:
            for page in range(len(pdf_reader.pages)):
                page_obj = pdf_reader.pages[page]
                text = page_obj.extract_text()
                lines = text.split('\n')
                for line in lines:
                    txt_file.write(line + '\n')

Example: Converting the pdf file 'poem.pdf' to text format.

In [30]:
convert_pdf_to_txt('poem.pdf', 'poem.txt')

3) Function that takes a text file and creates a new file containing the translation of the text from a specified source language to a specified target language.

In [44]:
def translated_file(input_file_path, src, tgt):
    
    with open(input_file_path, 'r') as f:
        text = f.readlines()
        
    translated_list = []

    for i in range(len(text)):
        translated_list.append(translator_object.translate(text[i], src, tgt))
    
    output_file_path = input_file_path.replace('.txt', '_translated.txt')    
    with open(output_file_path, 'w') as f:
        for i in range(len(text)):
            f.write(translated_list[i]+'\n') 

In [46]:
translated_file('poem.txt', 'fr', 'de')